# pvData Overview

# An aggregate data type
<br/>
Suppose want to encode a statistical summary of N floating point values.
Let’s say we want
1.	A mean (or some other sort average)
2.	The number of samples
3.	The standard deviation (or some other measure of spread)
4.	The maximum value
5.	The minimum value

We could encode this as structure consisting of an ordered set of fields.  
Each field has a name and a type.  
Let’s say:  
1.	A 64-bit floating point field for the average. Let’s call this type “double”. For the name of the field, we could choose “average”, but let’s call it value.
2.	A 64-bit integer field for the number of samples. Let’s call the type “long” and name the field “N”.
3.	A double field for the spread. Let’s call it “dispersion”
4.	A double field “max”
5.	A double field “min”



We can write a description of our type as follows:  
```
structure
    double value
    long N
    double dispersion
    double max
    double min
```

Let’s say we have 1000 samples with a mean of 2.0, a standard deviation of 1.0 and a minimum of 0.1 and a maximum of 4.4. We could our encoding of the statistical summary as:
 
```
structure
    double value       2.0
    long N            1000
    double dispersion  1.0 
    double max         4.4
    double min         0.1
```
This system of encoding data in EPICS V4 is called pvData and this way of writing pvData structures is called the pvData meta language.

Some jargon:
There are two types of objects in pvData: *Introspection objects* and *data objects*.

The first type of structure above, which describes the type of a piece of structured data, is an example of an *introspection object*.


The second structure, which carries values, is called a *data object*.
Structure data objects have an *introspection object* and some of the fields can carry data (as in the case above).

The values of the fields are referred to as the *value data* of the fields. The value data of a structure’s fields are called the *value data* of the structure.

The introspection object corresponding to a data object is referred to as its *introspection data* or its *introspection type*.

We can additionally give the type of the structure a name, called the type ID which helps identify the structure. The default type ID for a structure is **structure**.  

The above structure is an example of a standard structure (a Normative Type) called NTAggregate. So let’s give it the type ID **NTAggregate**.

When we write a structure we typically write it with the type ID:
```
NTAggregate
    double value
    long N
    double dispersion
    double max
    double min
```

and the data object above as
```
NTAggregate
    double value       2.0
    long N            1000
    double dispersion  1.0 
    double max         4.4
    double min         0.1
```

# Structure subfields
<br/>
Now suppose we want to add timestamps for the first and last sample in the data. We could encode the timestamp using the structure:
```
time_t
    long secondsPastEpoch
    int nanoseconds
    int userTag
```
where **int** denotes a 32-bit signed integer

We could then add structure subfields to our NTAggregate:
```
NTAggregate
    double value
    long N
    double dispersion
    double max
    double min
    time_t firstTimeStamp
        long secondsPastEpoch
        int nanoseconds
        int userTag
    time_t lastTimeStamp
        long secondsPastEpoch
        int nanoseconds
        int userTag
```

The two timestamp fields are fields of the top-level structure as well as the previous scalar fields.

Their introspection type is the **time_t** structure described above. These fields are themselves structures with fields secondsPastEpoch, nanoseconds and userTag.  

The indentation rules are similar to the indentation rules in Python.

An example of a corresponding data object would be: 
```
NTAggregate
    double value       2.0
    long N            1000
    double dispersion  1.0 
    double max         4.4
    double min         0.1
    time_t firstTimeStamp
        long secondsPastEpoch 1460589140
        int nanoseconds         11436967
        int userTag                    0
    time_t lastTimeStamp
        long secondsPastEpoch 1460589149
        int nanoseconds        889605397
        int userTag                    0
```

# pvData structures
<br/>
In general a pvData structure consists of
* 0 or more fields
* A type ID

<br/>
Each field has
* A name
* An introspection type

<br/>
For data objects a field may also have values.	

We write a structure as the type ID followed by 0 or more fields on subsequent lines, 1 per line, indented. The precise amount of indentation is not significant, only the level relative to other fields.

An example is the **NTAggregate** we’ve seen already:

```
NTAggregate
    double value
    long N
    double dispersion
    double max
    double min
```

# Field Types
<br/>
There are 6 basic field types namely

  * Scalars
  * Scalar arrays
  * Structures
  * Structure arrays
  * Union
  * Union arrays

<br/>
This field type is part of the introspection type of the field.

We’ve already seen examples of two of these: scalars and structures.

# Scalar Fields
<br/>
In general scalar fields can be one of the following scalar types
  * boolean
  * 8, 16, 32 or 64-bit signed integers: byte, short, int, long
  * 8, 16, 32 or 64-bit signed integers: ubyte, ushort, uint, ulong
  * 32 or 64-bit floating point numbers: float, double
  * string

<br/>
The scalar types are based on the Java primitive types with the addition of unsigned integers and string.

The scalar type entirely specifies the introspection type of a scalar field.

<sub-slide> 
For data objects, scalar fields additionally have value data.

The possible values for the data for each type are what you would expect. For example:

  * boolean: true or false
  * byte: integer in the range -128 to 127
  * ushort: integer in the range 0 to 65535
  * double: 64-bit (IEEE 754) floating point value (e.g 0.1, -3.5e-7, NaN)

The introspection type is written in the pvData meta language as
```
<scalar-type>
```
and scalar field as 
```
<scalar-type> <field-name>
```
<br/>
Examples:
```
double value
long N
string message
```

For data objects fields are written as
```
<scalar-type> <field-name> <value>
```
<br/>
Examples:
```
double value 3.14159
long N 1000
string message HIHI_ALARM
```

# Structure fields
<br/>
For an introspection object, a structure field, as for fields of all types, consists of introspection object and a field name.

For a structure field that introspection object is itself a structure.

In other words the top-level structure has a field whose introspection object is a structure.

As usual we write a structure with structure fields by added the fields one per line, indented, after the type ID.

For the structure fields we indent their fields one level further.
```
NTScalar
    double value
    alarm_t alarm
        int severity
        int status
        string message
    time_t timeStamp
        long secondsPastEpoch
        int nanoseconds
        int userTag
```

Data objects are similar except the fields of a structure field may have values.

```
NTScalar
    double value              8
    alarm_t alarm
        int severity          2
        int status            3
        string message        HIHI_ALARM
    time_t timeStamp
        long secondsPastEpoch 1460589140
        int nanoseconds       389605397
        int userTag           0
```

So in this case the top-level structure has a structure field
```
alarm_t alarm
    int severity
    int status
    string message
```
<br/>
whose name is **alarm** and whose introspection type is the structure
```
alarm_t
    int severity
    int status
    string message
```
<br/>
and a second structure field
```
time_t timeStamp
    long secondsPastEpoch
    int nanoseconds
    int userTag
```
<br/>
whose introspection type is the structure
```
time_t
    long secondsPastEpoch
    int nanoseconds
    int userTag
```

# Scalar Arrays
<br/>
Suppose we want to encode a table of archiver data for a PV whose value is double.

We want columns for the value, the seconds past UNIX epoch, nanoseconds and the alarm severities and statuses. We’ll also have a label for each column.

<sub-slide>
We could encode this as a structure with one field for each column. These fields will be array fields with elements of the appropriate type. We’ll also have an array of strings for the labels. To separate the columns from the labels we’ll put it in a sub-structure. Call it **value** as it’s the primary value of the structure:
```
NTTable
    string[] labels
    structure value
        double[] value
        long[] secondsPastEpoch
        int[] nanoseconds 
        int[] severity
        int[] status
```

**string[ ]** denotes string array field and so on.

<sub-slide>
An example of a corresponding data object is:
<pre>
NTTable
    string[] labels [value,  seconds, nanoseconds,  status, severity]
    structure value
        double[] value          [       1.1,       1.2,         2.0]
        long[] secondsPastEpoch [1460589140, 1460589141, 1460589142]
        int[] nanoseconds       [ 164235768,  164235245,  164235256]
        int[] severity          [         0,          0,          1]
        int[] status            [         0,          0,          3]
</pre>

<sub-slide>
In general scalar arrays fields are used to encode arrays of data of the same scalar type.

The possible scalar types are the same as for scalar fields:
  * boolean
  * 8, 16, 32 or 64-bit signed integers: byte, short, int, long
  * 8, 16, 32 or 64-bit signed integers: ubyte, ushort, uint, ulong
  * 32 or 64-bit floating point numbers
  * string

<br/>
Like scalars, the introspection type of a scalar array field is completely specified by the scalar type.

<sub-slide>
We write the introspection type in the meta language as
```
<scalar-type>[]
```
So the possible introspection types are
  * boolean[]
  * byte[], short[], int[], long[]
  * ubyte[], ushort[], uint[], ulong[]
  * float[], double[]
  * string[]

<br/>
Fields are written 
```
<scalar-type>[] <field-name>
```

<br/>
For example
```
string[] labels
double[] value
int[] status
```

The value data consists of zero or more scalar values. The set of possible values for each element is the same as for the values scalar field.
<br/>
We write these values as a comma separated list with square brackets.
<br/>
So examples of the data objects corresponding to the above fields are
```
string[] labels [value,  seconds, nanoseconds,  status, severity] 
double[] value  [1.1, 1.2, 2.0] 
int[] status    [  0,   0,   3]
```